In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from src import hrNetVideo
from library.GeneralOps import writeYUV420, runTerminalCmd
from src.auxFunctions import reconstruct, deconstruct, readFrames
from library import hrNet
import tensorflow as tf
from src.rgb2yuv_yuv2rgb import RGB2YUV

In [ ]:
# baseRefPath = '/data/nwaliv/trainVideoSetRef/'
# baseCompPath = '/data/nwaliv/trainVideoSetDeg/'
baseRefPath = '/ref/'
baseCompPath = '/alpha0.01/'

DATAFRAME = pd.read_csv('dataFrames/trainVideoSet_perframeCambi_maxFrame.csv')
DATAFRAME = DATAFRAME[DATAFRAME['FrameNum'] != 0]
DATAFRAME = DATAFRAME[DATAFRAME['FrameNum'] != DATAFRAME['MaxFrame']]
DATAFRAME = DATAFRAME.reset_index(drop=True)
SAMPLE = DATAFRAME.sample(n=5, replace=False,random_state=10)
SAMPLE = SAMPLE.reset_index(drop=True)

for idx, _row in SAMPLE.iterrows():
    # get height/width of original video
    _height = _row['Height']
    _width = _row['Width']
    _frameNum = _row['FrameNum']
    _comp = os.path.join(baseCompPath,'res_'+ _row['Comp'])
    _ref = os.path.join(baseRefPath, _row['Ref'])
    print(_comp)

    
    #hrNetModel = hrNetVideo(_comp,(_width,_height),_row['MaxFrame'],modelWeights='modelWeights/287_HRNET.h5')
    #resFrame = hrNetModel.restore_frame(t=1)
    #_ = hrNetModel.restore_video()
    # instead change save video to return yuv frames instead and save outside of function
    #writeYUV420('output'+_comp,_,_,_,downsample=True)

In [ ]:
alphas = [1.0, 0.01, 0.001, 0.0001, 1e-05, 5e-06, 2.5e-06, 1.5e-06 ,1e-06]
for _alpha in alphas:
    baseRefPath = '/ref/'
    baseCompPath = f'/alpha{_alpha}/'
    df = pd.read_json(f"results/HRNET_CAMBI_ALPHA{_alpha}.json")

    DATAFRAME = pd.read_csv('dataFrames/trainVideoSet_perframeCambi_maxFrame.csv')
    DATAFRAME = DATAFRAME[DATAFRAME['FrameNum'] != 0]
    DATAFRAME = DATAFRAME[DATAFRAME['FrameNum'] != DATAFRAME['MaxFrame']]
    DATAFRAME = DATAFRAME.reset_index(drop=True)
    SAMPLE = DATAFRAME.sample(n=10, replace=False,random_state=0)
    SAMPLE = SAMPLE.reset_index(drop=True)

    df_peralpha = {
    'Ref': [],
    'Comp': [],
    'Height': [],
    'Width': [],
    'PSNR_Y': [],
    'PSNR_Cb': [],
    'PSNR_Cr': [],
    'PSNR_Y': [],
    'SSIM': [],
    'VMAF': [],
    'CAMBI': []
    }

    for idx, _row in SAMPLE.iterrows():
        # get height/width of original video
        _height = _row['Height']
        _width = _row['Width']
        _frameNum = _row['FrameNum']
        _comp = os.path.join(baseCompPath,'res_'+_row['Comp'])
        _ref = os.path.join(baseRefPath, _row['Ref'])
        #print(_comp)
        runTerminalCmd(f"vmaf -r {_ref} -d {_comp} -w {_width} -h {_height} -p 420 -b 8 --csv -o processing/evals.csv --feature psnr --feature float_ssim --feature cambi -q")
        cambi = pd.read_csv('processing/evals.csv')['cambi'].mean()
        psnr_y = pd.read_csv('processing/evals.csv')['psnr_y'].mean()
        psnr_cb = pd.read_csv('processing/evals.csv')['psnr_cb'].mean()
        psnr_cr = pd.read_csv('processing/evals.csv')['psnr_cr'].mean()
        ssim = pd.read_csv('processing/evals.csv')['ssim'].mean()
        vmaf = pd.read_csv('processing/evals.csv')['vmaf'].mean()
        df_peralpha['Ref'].append(_ref)
        df_peralpha['Comp'].append('res'+_comp)
        df_peralpha['Height'].append(_height)
        df_peralpha['Width'].append(_width)
        df_peralpha['PSNR_Y'].append(_psnr_y)
        df_peralpha['PSNR_Cb'].append(_psnr_cb)
        df_peralpha['PSNR_Cr'].append(_psnr_cr)
        df_peralpha['SSIM'].append(_ssim)
        df_peralpha['VMAF'].append(_vmaf)
        df_peralpha['CAMBI'].append(_cambi)
    
    df_peralpha = pd.DataFrame(df_peralpha)
    df_peralpha.to_csv(f'alpha{_alpha}/alphaScores.csv')

In [ ]:
import pandas as pd
import numpy as np
import os
from src import hrNet
from src.auxFunctions import reconstruct, deconstruct, readFrames
import tensorflow as tf
from library.GeneralOps import writeYUV420
from src.rgb2yuv_yuv2rgb import RGB2YUV

baseRefPath = '/data/nwaliv/trainVideoSetRef/'
baseCompPath = '/data/nwaliv/trainVideoSetDeg/'
alphas = [0.01, 0.001, 0.0001, 1e-05, 5e-06, 2.5e-06, 1.5e-06 ,1e-06]
generator = hrNet(2, [32, 64, 128, 256], 5).model()
for _alpha in alphas:
    print("alpha = ", _alpha)
    df = pd.read_json(f"results/HRNET_CAMBI_ALPHA{_alpha}.json")
    generator.load_weights(f'results/HRNET_CAMBI_ALPHA{_alpha}.h5')

    DATAFRAME = pd.read_csv('dataFrames/trainVideoSet_perframeCambi_maxFrame.csv')
    DATAFRAME = DATAFRAME[DATAFRAME['FrameNum'] != 0]
    DATAFRAME = DATAFRAME[DATAFRAME['FrameNum'] != DATAFRAME['MaxFrame']]
    DATAFRAME = DATAFRAME.reset_index(drop=True)
    SAMPLE = DATAFRAME.sample(n=5, replace=False,random_state=10)
    SAMPLE = SAMPLE.reset_index(drop=True)

    for idx, _row in SAMPLE.iterrows():
        # get height/width of original video
        _height = _row['Height']
        _width = _row['Width']
        _frameNum = _row['FrameNum']
        _comp = os.path.join(baseCompPath, _row['Comp'])
        _ref = os.path.join(baseRefPath, _row['Ref'])
        print(_comp)

        restoredY = np.empty((_row['MaxFrame'],_height,_width))
        restoredU = np.empty((_row['MaxFrame'],_height,_width))
        restoredV = np.empty((_row['MaxFrame'],_height,_width))
        restoredFrames = np.empty((_row['MaxFrame'],_height,_width,3))

        for frame in range(_row['MaxFrame']):
            print("Frame ", frame)
            frame_tmin1,frame_t,frame_tplus1 = readFrames(_comp,(_width,_height),frame,_row['MaxFrame'])

            window_tmin1 = deconstruct(frame_tmin1,192)
            window_t = deconstruct(frame_t,192)
            window_tplus1 = deconstruct(frame_tplus1,192)

            inputPatches = np.concatenate((window_tmin1, window_t, window_tplus1), axis=-1)
            inputPatches = np.expand_dims(inputPatches, axis=0)
            numPatches = inputPatches.shape[1]
            inputPatches = tf.convert_to_tensor(inputPatches)

            outputPatches = np.empty((1,numPatches,192,192,3))
            for patch in range(numPatches):
                outputPatches[:,patch,:,:,:] = generator(inputPatches[:,patch,:,:,:],training=False)

            outputFrame = reconstruct(outputPatches[0],frame_t,192)
            restoredFrames[frame,:,:,:] = outputFrame[:,:,:]

            outputFrameYUV = RGB2YUV(outputFrame)

            Yout = outputFrameYUV[:,:,0]
            Uout = outputFrameYUV[:,:,1]
            Vout = outputFrameYUV[:,:,2]

            restoredY[frame,:,:] = Yout
            restoredU[frame,:,:] = Uout
            restoredV[frame,:,:] = Vout

        savePath = f'alpha{(_alpha)}/'   
        writeYUV420(savePath+"res_" +_row['Comp'],np.uint8(restoredY),np.uint8(restoredU),np.uint8(restoredV),downsample=True)

In [ ]:
baseRefPath = '/bandon/nwaliv/testVideoSet/'
baseCompPath = '/bandon/nwaliv/adabandVideos/'


df_adaband = {
    'Ref': [],
    'Comp': [],
    'Height': [],
    'Width': [],
    'PSNR_Y': [],
    'PSNR_Cb': [],
    'PSNR_Cr': [],
    'PSNR_Y': [],
    'SSIM': [],
    'VMAF': [],
    'CAMBI': []
    }

DATAFRAME = pd.read_csv('dataFrames/testVideoSet.csv')
for _idx, _row in DATAFRAME.iterrows():
    _height = _row['Height']
    _width = _row['Width']
    _comp = os.path.join(baseCompPath,'adaband_'+_row['Comp'])
    _ref = os.path.join(baseRefPath, _row['Ref'])
    runTerminalCmd(f"vmaf -r {_ref} -d {_comp} -w {_width} -h {_height} -p 420 -b 8 --csv -o processing/evals.csv --feature psnr --feature float_ssim --feature cambi -q")
    cambi = pd.read_csv('processing/evals.csv')['cambi'].mean()
    psnr_y = pd.read_csv('processing/evals.csv')['psnr_y'].mean()
    psnr_cb = pd.read_csv('processing/evals.csv')['psnr_cb'].mean()
    psnr_cr = pd.read_csv('processing/evals.csv')['psnr_cr'].mean()
    ssim = pd.read_csv('processing/evals.csv')['ssim'].mean()
    vmaf = pd.read_csv('processing/evals.csv')['vmaf'].mean()
    df_adaband['Ref'].append(_ref)
    df_adaband['Comp'].append('res'+_comp)
    df_adaband['Height'].append(_height)
    df_adaband['Width'].append(_width)
    df_adaband['PSNR_Y'].append(_psnr_y)
    df_adaband['PSNR_Cb'].append(_psnr_cb)
    df_adaband['PSNR_Cr'].append(_psnr_cr)
    df_adaband['SSIM'].append(_ssim)
    df_adaband['VMAF'].append(_vmaf)
    df_adaband['CAMBI'].append(_cambi)
    
df_adaband = pd.DataFrame(df_adaband)
df_adaband.to_csv('dataFrames/adabandScores.csv')

In [ ]:
import pandas as pd
import os
from library.GeneralOps import runTerminalCmd

alphas = [1.0, 0.01, 0.001, 0.0001, 1e-05, 5e-06, 2.5e-06, 1.5e-06 ,1e-06]
for _alpha in alphas:
    baseRefPath = 'ref/'
    baseCompPath = f'alpha{_alpha}/'
    df = pd.read_json(f"results/HRNET_CAMBI_ALPHA{_alpha}.json")

    DATAFRAME = pd.read_csv('dataFrames/trainVideoSet_perframeCambi_maxFrame.csv')
    DATAFRAME = DATAFRAME[DATAFRAME['FrameNum'] != 0]
    DATAFRAME = DATAFRAME[DATAFRAME['FrameNum'] != DATAFRAME['MaxFrame']]
    DATAFRAME = DATAFRAME.reset_index(drop=True)
    SAMPLE = DATAFRAME.sample(n=5, replace=False,random_state=10)
    SAMPLE = SAMPLE.reset_index(drop=True)

    df_peralpha = {
    'Ref': [],
    'Comp': [],
    'Height': [],
    'Width': [],
    'PSNR_Y': [],
    'PSNR_Cb': [],
    'PSNR_Cr': [],
    'PSNR_Y': [],
    'SSIM': [],
    'VMAF': [],
    'CAMBI': []
    }

    for idx, _row in SAMPLE.iterrows():
        # get height/width of original video
        _height = _row['Height']
        _width = _row['Width']
        _comp = os.path.join(baseCompPath,'res_'+_row['Comp'])
        _ref = os.path.join(baseRefPath, _row['Ref'])
        #print(_comp)
        runTerminalCmd(f"vmaf -r {_ref} -d {_comp} -w {_width} -h {_height} -p 420 -b 8 --csv -o processing/evals.csv --feature psnr --feature float_ssim --feature cambi -q")
        _cambi = pd.read_csv('processing/evals.csv')['cambi'].mean()
        _psnr_y = pd.read_csv('processing/evals.csv')['psnr_y'].mean()
        _psnr_cb = pd.read_csv('processing/evals.csv')['psnr_cb'].mean()
        _psnr_cr = pd.read_csv('processing/evals.csv')['psnr_cr'].mean()
        _ssim = pd.read_csv('processing/evals.csv')['float_ssim'].mean()
        _vmaf = pd.read_csv('processing/evals.csv')['vmaf'].mean()
        df_peralpha['Ref'].append(_row['Ref'])
        df_peralpha['Comp'].append('res_'+_row['Comp'])
        df_peralpha['Height'].append(_height)
        df_peralpha['Width'].append(_width)
        df_peralpha['PSNR_Y'].append(_psnr_y)
        df_peralpha['PSNR_Cb'].append(_psnr_cb)
        df_peralpha['PSNR_Cr'].append(_psnr_cr)
        df_peralpha['SSIM'].append(_ssim)
        df_peralpha['VMAF'].append(_vmaf)
        df_peralpha['CAMBI'].append(_cambi)
    df_peralpha = pd.DataFrame(df_peralpha)
    df_peralpha.to_csv(f'alpha{_alpha}/alphaScores.csv')